# Licence and access rights notebook

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %load imports.py
# Basic imports
import os
import sys

import matplotlib

sys.path.append('./')

# SQL database
import pymysql

# Classical external libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

# Import py files for generic functions
from sql import *
from helpers import *


## Connection to DB

In [3]:
# Connexion database
engine = db_engine()

## Load Useful Data

In [4]:
newspapers_df = read_table('impresso.newspapers', engine)
issues_df = read_table('impresso.issues', engine)
# Create new decade column
issues_df['decade'] = issues_df.apply(lambda row: row.year-row.year%10, axis = 1) 

In [5]:
newspapers_metadata_df = read_table('newspapers_metadata', engine)
meta_properties_df = read_table('meta_properties', engine)
PROPERTIES = meta_properties_df.name.unique()

## Number of issues by access right policy

In [6]:
issues_df.access_rights.unique()

array(['Closed', 'OpenPublic', 'OpenPrivate'], dtype=object)

In [7]:
issues_df.head(3)

,id,year,month,day,edition,access_rights,created,last_modified,is_damaged,s3_version,newspaper_id,decade
0,actionfem-1927-10-15-a,1927,10,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem,1920
1,actionfem-1927-11-15-a,1927,11,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem,1920
2,actionfem-1927-12-15-a,1927,12,15,a,Closed,2019-06-15 12:22:38,NaT,0,,actionfem,1920


In [8]:
count_issue_ar_df, _, _ = group_and_count(issues_df, ['access_rights'], 'id', False)

In [9]:
total_nb_issues = count_issue_ar_df['count'].sum()
count_issue_ar_df['rate'] = count_issue_ar_df['count']/total_nb_issues

In [10]:
count_issue_ar_df

,access_rights,count,rate
0,Closed,168685,0.381760
1,OpenPrivate,225703,0.510801
2,OpenPublic,47473,0.107439


On Oct. 15th 2019 : there are 225'703 open-private issues, 168'685 closed issues, and 47'473 open-public issues in the impresso db. <br/>
In other words, half issues (~51%) are open-private, around 38% are closed and finally only almost 11% of issues are open-public.

### Filter on property

#### By country

In [11]:
newspapers_select_ch = np_by_property(newspapers_metadata_df, meta_properties_df, 'countryCode', 'CH')
issues_df_select_ch = filter_df_by_np_id(issues_df, newspapers_select_ch)

newspapers_select_lux = np_by_property(newspapers_metadata_df, meta_properties_df, 'countryCode', 'LU')
issues_df_select_lux = filter_df_by_np_id(issues_df, newspapers_select_lux)

In [12]:
count_issue_ar_ch_df, _, _ = group_and_count(issues_df_select_ch, ['access_rights'], 'id', False)
count_issue_ar_lux_df, _, _ = group_and_count(issues_df_select_lux, ['access_rights'], 'id', False)

In [13]:
total_nb_issues_ch = count_issue_ar_ch_df['count'].sum()
total_nb_issues_lux = count_issue_ar_lux_df['count'].sum()

In [14]:
total_nb_issues_ch_lux = total_nb_issues_ch+total_nb_issues_lux

In [15]:
count_issue_ar_ch_df['ch_rate'] = count_issue_ar_ch_df['count']/total_nb_issues_ch

In [16]:
count_issue_ar_lux_df['lux_rate'] = count_issue_ar_lux_df['count']/total_nb_issues_lux

In [17]:
count_issue_ar_ch_df

,access_rights,count,ch_rate
0,Closed,112676,0.316981
1,OpenPrivate,225611,0.634691
2,OpenPublic,17179,0.048328


In [18]:
count_issue_ar_lux_df

,access_rights,count,lux_rate
0,Closed,56009,0.653814
1,OpenPublic,29656,0.346186


Looking at the country level is very interesting because we see that the access right of issues in both countries are very different.
- Switzerland : The percentage of open public issues is very low. Most issued (~63%) are open-private and the large majority of the rest (~32%) has closed access rights.
- Luxembourg : On the contrary, there are no open-private access right on issues. Approximately two thirds (65%) of the issues has closed access rights, and the last third has open-public access.

### Issues access rights at the level of newspapers

#### Can several issues of a same newspaper have different access right levels ?

In [19]:
count_issues_np_ar = issues_df.groupby(['newspaper_id', 'access_rights']).id.count().reset_index(name="count")
count_issues_np_ar

,newspaper_id,access_rights,count
0,BDC,OpenPublic,23
1,CDV,OpenPublic,1200
2,DLE,OpenPrivate,402
3,EDA,OpenPublic,544
4,EXP,OpenPrivate,46196
5,EXP,OpenPublic,8
6,GDL,OpenPrivate,51062
7,GDL,OpenPublic,8
8,IMP,OpenPrivate,40227
9,IMP,OpenPublic,1


In [20]:
# Count number of different access rights per newspapers and mark the ones which have more that 1 access right policy
nb_ar_np = (issues_df.groupby('newspaper_id')['access_rights'].nunique() > 1).reset_index(name='value')

In [21]:
# Get ids of the newspapers which have several access right levels
np_multiple_ar = nb_ar_np[nb_ar_np['value']].newspaper_id.unique()

In [22]:
print("The following papers have issues with different access rights : ")
np_multiple_ar

The following papers have issues with different access rights : 


array(['EXP', 'GDL', 'IMP', 'LCE', 'LSE', 'indeplux', 'luxwort',
       'waeschfra'], dtype=object)

#### Insights on the newspapers which have several access right levels

In [31]:
# Keep newspaper_id, access_right and count info on np with multiple access right levels
multiple_ar_np_df = filter_df_by_np_id(count_issues_np_ar, np_multiple_ar)

# Pivot to get a count per access right level & set the index correctly
multiple_ar_np_df = multiple_ar_np_df.pivot_table('count', ['newspaper_id'], 'access_rights')
multiple_ar_np_df.reset_index(drop=False, inplace=True)

# Change NaN values to zeros
multiple_ar_np_df = multiple_ar_np_df.fillna(0)

# Compute the rate of access right per level per np
multiple_ar_np_df['Total'] = multiple_ar_np_df[['Closed', 'OpenPrivate', 'OpenPublic']].sum(axis=1)
multiple_ar_np_df['rate_Closed'] = multiple_ar_np_df['Closed']/multiple_ar_np_df['Total']
multiple_ar_np_df['rate_OpenPrivate'] = multiple_ar_np_df['OpenPrivate']/multiple_ar_np_df['Total']
multiple_ar_np_df['rate_OpenPublic'] = multiple_ar_np_df['OpenPublic']/multiple_ar_np_df['Total']
multiple_ar_np_df

access_rights,newspaper_id,Closed,OpenPrivate,OpenPublic,Total,rate_Closed,rate_OpenPrivate,rate_OpenPublic
0,EXP,0.0,46196.0,8.0,46204.0,0.000000,0.999827,0.000173
1,GDL,0.0,51062.0,8.0,51070.0,0.000000,0.999843,0.000157
2,IMP,0.0,40227.0,1.0,40228.0,0.000000,0.999975,0.000025
3,LCE,0.0,16291.0,1.0,16292.0,0.000000,0.999939,0.000061
4,LSE,0.0,20182.0,1.0,20183.0,0.000000,0.999950,0.000050
5,indeplux,16447.0,0.0,1903.0,18350.0,0.896294,0.000000,0.103706
6,luxwort,16027.0,0.0,14328.0,30355.0,0.527986,0.000000,0.472014
7,waeschfra,224.0,0.0,436.0,660.0,0.339394,0.000000,0.660606


Papers which have issues with several access rights can be grouped into three groups :
- Group 1 : Those for which almost 100% of their issues have the same access right level and for which other access right permissions are very rare. It is actually the case for all swiss newspapers which have several access rights : EXP, GDL, IMP, LCE and LSE. For the last three, only one issue among all has a different access right. We could investigate on whether this is an outlier and what it is due to. Note also that they all have the same access right majority : OpenPrivate (and minority : OpenPublic).
- Group 2 : Those for which most of their issues have the same access right level. It is the case for indeplux : almost 90% of its issues have access right Closed, whereas the left 10% are OpenPublic.
- Group 3 : Those for which the difference between the two access right levels is more balanced. It is the case for luxwort (53% closed vs 47% open public) and waeschfra (34% closed vs 66% open public). 

Based on these observations, we can investigate on the following questions :
- Where to the rare values in group 1 come from ? Errors from the dataset ? Particular issue-s ?
- Are the access right levels in group 2 and 3 linked to some other property of the issues or to the date in time ?

In [32]:
# TODO next : see if the access right levels have a link with dates for thoses np

## Number of content items by access right policy, filtered by property